# Build Text to SQL with Cortex Analyst

In this notebook we show how to build a text-to-sql app using Cortex Analyst.

Let's start with trying on our own.

Can we easily just call an LLM, give it our user query and some table information, and expect it to provide us usable SQL?

Let's try.

In [ ]:
from snowflake.cortex import Complete

user_query = "What is the highest daily revenue recorded in a single day in each sales region?"

naive_text_to_sql_messages = [
        {
            'role': 'system',
            'content': 'You are a helpful assistant that writes SQL to answer natural language questions. Respond with only SQL'
        },
        {
            'role': 'user',
            'content': f'The user has posed a question, which is captured in: {user_query}. '
                       f'All tables are in the fully qualified snowflake schema: CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.'
                       f'The tables available that can be used by the SQL are:'
                       f'table: daily_revenue. columns: date, product_id, region_id, revenue, cogs, forecasted_revenue, daily_forecasted_revenue, daily_profit'
                       f'table: product. columns: product_id, product_line'
                       f'table: region. region_id, region, state'
                       f'Please write valid snowflake SQL to answer the user question.'
        }
    ]
    

sql = Complete("claude-3-5-sonnet", naive_text_to_sql_messages)

In [ ]:
sql

In [ ]:
session = get_active_session()

sql_output = session.sql(sql)

sql_output

That was an easy question and we wanted to use just one table, and we still struggled to quickly come up with the needed context and prompt engineering to generate useful SQL with a SOTA LLM. Now imagine harder questions, hundreds of tables, and the need to perform complicated JOINS.

Enter: Cortex Analyst

## Create the required roles to use Cortex Analyst

Make sure to add your username in Line 10!

In [ ]:
USE WAREHOUSE S;

USE ROLE SECURITYADMIN;

CREATE OR REPLACE ROLE cortex_user_role;
GRANT DATABASE ROLE SNOWFLAKE.CORTEX_USER TO ROLE cortex_user_role;

USE ROLE SECURITYADMIN;

GRANT ROLE cortex_user_role TO USER <user>;

## Create the required databases, schemas and warehouse for this example

In [ ]:
USE ROLE ACCOUNTADMIN;

-- Create demo database
CREATE OR REPLACE DATABASE cortex_analyst_demo;

-- Create schema
CREATE OR REPLACE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Create warehouse
CREATE OR REPLACE WAREHOUSE cortex_analyst_wh
    WAREHOUSE_SIZE = 'large'
    WAREHOUSE_TYPE = 'standard'
    AUTO_SUSPEND = 60
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
COMMENT = 'Warehouse for Cortex Analyst demo';

GRANT USAGE ON WAREHOUSE cortex_analyst_wh TO ROLE cortex_user_role;
GRANT OPERATE ON WAREHOUSE cortex_analyst_wh TO ROLE cortex_user_role;

GRANT OWNERSHIP ON SCHEMA cortex_analyst_demo.revenue_timeseries TO ROLE cortex_user_role;
GRANT OWNERSHIP ON DATABASE cortex_analyst_demo TO ROLE cortex_user_role;

USE ROLE cortex_user_role;

-- Use the created warehouse
USE WAREHOUSE cortex_analyst_wh;

USE DATABASE cortex_analyst_demo;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Create stage for raw data
CREATE OR REPLACE STAGE raw_data DIRECTORY = (ENABLE = TRUE);

/*--
• Fact and Dimension Table Creation
--*/

-- Fact table: daily_revenue
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.daily_revenue (
    date DATE,
    revenue FLOAT,
    cogs FLOAT,
    forecasted_revenue FLOAT,
    product_id INT,
    region_id INT
);

-- Dimension table: product_dim
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.product_dim (
    product_id INT,
    product_line VARCHAR(16777216)
);

-- Dimension table: region_dim
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.region_dim (
    region_id INT,
    sales_region VARCHAR(16777216),
    state VARCHAR(16777216)
);

There are three data files and one YAML file included in the this repository.

daily_revenue.csv
region.csv
product.csv
revenue_timeseries.yaml
You will now upload these files to your Snowflake account and ingest the data files into the tables created in the previous step.

To upload the data files:

1. Set your role to Cortex User Role
2. Navigate to the Data tab in Snowsight, and select Add Data
3. On the Add Data page, select Load files into a stage
4. Select the four files that you want to upload (listed above)
5. Select CORTEX_ANALYST_DEMO as Database, REVENUE_TIMESERIES as Schema, and RAW_DATA as Stage
6. Click Upload

Let's go check that the files were successfully uploaded to the stage. In the Snowsight UI:

7. Select Data » Databases
8. Select the CORTEX_ANALYST_DEMO database and REVENUE_TIMESERIES Schema that contain the stage
9. Select Stages and select the RAW_DATA stage
10. If prompted, select Enable Directory Table and the CORTEX_ANALYST_WH to refresh the directory table

You should see the four files listed in the stage.

## Copy data from stage into tables

In [ ]:
USE ROLE CORTEX_USER_ROLE;
USE DATABASE CORTEX_ANALYST_DEMO;
USE SCHEMA CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES;
USE WAREHOUSE CORTEX_ANALYST_WH;

In [ ]:
/*--
• looad data into tables
--*/

USE ROLE CORTEX_USER_ROLE;
USE DATABASE CORTEX_ANALYST_DEMO;
USE SCHEMA CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES;
USE WAREHOUSE CORTEX_ANALYST_WH;

COPY INTO CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.DAILY_REVENUE
FROM @raw_data
FILES = ('daily_revenue.csv')
FILE_FORMAT = (
    TYPE=CSV,
    SKIP_HEADER=1,
    FIELD_DELIMITER=',',
    TRIM_SPACE=FALSE,
    FIELD_OPTIONALLY_ENCLOSED_BY=NONE,
    REPLACE_INVALID_CHARACTERS=TRUE,
    DATE_FORMAT=AUTO,
    TIME_FORMAT=AUTO,
    TIMESTAMP_FORMAT=AUTO
    EMPTY_FIELD_AS_NULL = FALSE
    error_on_column_count_mismatch=false
)

ON_ERROR=CONTINUE
FORCE = TRUE ;



COPY INTO CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.PRODUCT_DIM
FROM @raw_data
FILES = ('product.csv')
FILE_FORMAT = (
    TYPE=CSV,
    SKIP_HEADER=1,
    FIELD_DELIMITER=',',
    TRIM_SPACE=FALSE,
    FIELD_OPTIONALLY_ENCLOSED_BY=NONE,
    REPLACE_INVALID_CHARACTERS=TRUE,
    DATE_FORMAT=AUTO,
    TIME_FORMAT=AUTO,
    TIMESTAMP_FORMAT=AUTO
    EMPTY_FIELD_AS_NULL = FALSE
    error_on_column_count_mismatch=false
)

ON_ERROR=CONTINUE
FORCE = TRUE ;



COPY INTO CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.REGION_DIM
FROM @raw_data
FILES = ('region.csv')
FILE_FORMAT = (
    TYPE=CSV,
    SKIP_HEADER=1,
    FIELD_DELIMITER=',',
    TRIM_SPACE=FALSE,
    FIELD_OPTIONALLY_ENCLOSED_BY=NONE,
    REPLACE_INVALID_CHARACTERS=TRUE,
    DATE_FORMAT=AUTO,
    TIME_FORMAT=AUTO,
    TIMESTAMP_FORMAT=AUTO
    EMPTY_FIELD_AS_NULL = FALSE
    error_on_column_count_mismatch=false
)

ON_ERROR=CONTINUE
FORCE = TRUE ;

## Create a Cortex Search service to help Analyst

Now, you will integrate Cortex Search as a way to improve literal string searches to help Cortex Analyst generate more accurate SQL queries. Writing the correct SQL query to answer a question sometimes requires knowing exact literal values to filter on. Since those values can't always be extracted directly from the question, a search of some kind may be needed.

In [ ]:
USE DATABASE cortex_analyst_demo;
USE SCHEMA revenue_timeseries;
use ROLE cortex_user_role;

In [ ]:
USE DATABASE cortex_analyst_demo;
USE SCHEMA revenue_timeseries;
use ROLE cortex_user_role;

CREATE OR REPLACE CORTEX SEARCH SERVICE product_line_search_service
  ON product_dimension
  WAREHOUSE = cortex_analyst_wh
  TARGET_LAG = '1 hour'
  AS (
      SELECT DISTINCT product_line AS product_dimension FROM product_dim
  );

## Using 'semantic models' to share context with LLMs

Semantic models give us a structured way to capture information about our data and how it relates to our business. We can pass this information to the LLM to get better SQL generation.

adsf

In [ ]:
semantic_model = """"""

semantic_model_name = """
name: Revenue
"""

semantic_model += semantic_model_name

semantic_model_revenue = """
tables:
  - name: daily_revenue
    description: Daily total revenue, aligned with daily "Cost of Goods Sold" (COGS), and forecasted revenue.
    base_table:
      database: cortex_analyst_demo
      schema: revenue_timeseries
      table: daily_revenue
    primary_key:
      columns:
        - date
        - product_id
        - region_id
    dimensions:
      - name: product_id
        expr: product_id
        data_type: number
      - name: region_id
        expr: region_id
        data_type: number
    time_dimensions:
      - name: date
        expr: date
        description: date with measures of revenue, COGS, and forecasted revenue.
        unique: true
        data_type: date
    measures:
      - name: daily_revenue
        expr: revenue
        description: total revenue for the given day
        synonyms: ["sales", "income"]
        default_aggregation: sum
        data_type: number
      - name: daily_cogs
        expr: cogs
        description: total cost of goods sold for the given day
        synonyms: ["cost", "expenditures"]
        default_aggregation: sum
        data_type: number
      - name: daily_forecasted_revenue
        expr: forecasted_revenue
        description: total forecasted revenue for a given day
        synonyms: ["forecasted sales", "forecasted income"]
        default_aggregation: sum
        data_type: number
      - name: daily_profit
        description: profit is the difference between revenue and expenses.
        expr: revenue - cogs
        data_type: number
      - name: daily_forecast_abs_error
        synonyms:
          - absolute error
          - L1
        description: absolute error between forecasted and actual revenue
        expr: abs(forecasted_revenue - revenue)
        data_type: number
        default_aggregation: avg
"""

semantic_model += semantic_model_revenue

Now that we've expended more effort in writing down our semantic model of the data we want to use, let's try again to 'naively' pass it to a SOTA LLM.

In [ ]:
from snowflake.cortex import Complete

user_query = "What is the highest daily revenue recorded in a single day in each sales region?"

naive_text_to_sql_messages = [
        {
            'role': 'system',
            'content': 'You are a helpful assistant that writes snowflake SQL to answer natural language questions. Respond with only SQL'
        },
        {
            'role': 'user',
            'content': f'The user has posed a question, which is captured in: {user_query}. '
                       f'All tables are in the fully qualified snowflake schema: CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.'
                       f'{semantic_model}'
                       f'Please write valid snowflake SQL to answer the user question, and do not include fences.'
        }
    ]
    

sql = Complete("claude-3-5-sonnet", naive_text_to_sql_messages)

In [ ]:
sql

In [ ]:
sql_output = session.sql(sql)

sql_output

This worked pretty well, but it'd be great to have the region name instead of just the ID.

Let's expand the semantic model to include the region table and how to join it with the revenue table.

In [ ]:
semantic_model_region = """
  - name: region
    description: Region dimension table with unique region identifiers and geographic attributes.
    base_table:
      database: cortex_analyst_demo
      schema: revenue_timeseries
      table: region_dim
    primary_key:
      columns:
        - region_id
    dimensions:
      - name: region_id
        expr: region_id
        data_type: number
      - name: sales_region
        expr: sales_region
        description: Region associated with revenue
        data_type: varchar
        sample_values:
          - North America
          - Europe
          - Asia
          - South America
          - Africa
"""

semantic_model += semantic_model_region

In [ ]:
semantic_model_relationships = """
relationships:
  - name: revenue_to_region
    left_table: daily_revenue
    right_table: region
    relationship_columns:
      - left_column: region_id
        right_column: region_id
    join_type: left_outer
    relationship_type: many_to_one
"""

semantic_model += semantic_model_relationships

In [ ]:
from snowflake.cortex import Complete

user_query = "What is the highest daily revenue recorded in a single day in each sales region?"

naive_text_to_sql_messages = [
        {
            'role': 'system',
            'content': 'You are a helpful assistant that writes snowflake SQL to answer natural language questions. Respond with only SQL'
        },
        {
            'role': 'user',
            'content': f'The user has posed a question, which is captured in: {user_query}. '
                       f'All tables are in the fully qualified snowflake schema: CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.'
                       f'{semantic_model}'
                       f'Please write valid snowflake SQL to answer the user question, and do not include fences.'
        }
    ]
    

sql = Complete("claude-3-5-sonnet", naive_text_to_sql_messages)

sql

In [ ]:
sql_output = session.sql(sql)

sql_output

Now that we've got a tougher task, even the SOTA LLM fails to generate valid SQL.

Let's turn to Cortex Analyst using the same semantic model.

## Call Cortex Analyst

First, we'll just show how to make the REST API call and return SQL

In [ ]:
import json
import requests
from typing import List
import _snowflake
from snowflake.snowpark.context import get_active_session

# Change this to match your semantic model stage/path
DEFAULT_SEMANTIC_MODEL = "CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.RAW_DATA/revenue_timeseries.yaml"

API_ENDPOINT = "/api/v2/cortex/analyst/message"
API_TIMEOUT = 50000  # in milliseconds

def get_sql_from_cortex_analyst(query: str) -> List[str]:
    """
    Calls Cortex Analyst with the user's query and returns any generated SQL statements.
    
    Returns:
        A list of SQL statements found in the Analyst's response (there can be more than one).
    """
    # Build the message list (only one user message for simplicity).
    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": query}],
        }
    ]
    
    # Build the request body
    request_body = {
        "messages": messages,
        "semantic_model": semantic_model,
    }
    
    # Make the request to the Analyst API
    resp = _snowflake.send_snow_api_request(
        "POST",  # method
        API_ENDPOINT,  # path
        {},  # headers
        {},  # params
        request_body,  # body
        None,  # request_guid
        API_TIMEOUT,  # timeout in milliseconds
    )
    
    # Use resp.json() to get parsed JSON
    parsed_content = json.loads(resp["content"])

    return parsed_content["message"]["content"][1]

user_query = "What is the highest daily revenue recorded in a single day in each sales region?"
analyst_output = get_sql_from_cortex_analyst(user_query)

In [ ]:
analyst_output

## Run the SQL

Next, we can run the sql and save it to a variable

In [ ]:
session = get_active_session()

sql_output = session.sql(analyst_output['statement'])

sql_output

## Let's not stop at the table, and get to a natural language response

Pass SQL output to LLM-readable markdown

Note: Make sure `tabulate` library is installed

In [ ]:
markdown_sql_output = sql_output.to_pandas().to_markdown(index=False)

In [ ]:
markdown_sql_output

## Send the query and SQL output to the LLM

In [ ]:
from snowflake.cortex import Complete
import pandas as pd

messages = [
        {
            'role': 'system',
            'content': 'You are a helpful assistant that sql output to answer natural language questions.'
        },
        {
            'role': 'user',
            'content': f'The user has posed a question, which is captured in: {user_query}. '
                       f'The question has been translated into a SQL statement, executed, and the results are found in:'
                       f'\n{markdown_sql_output}'
                       f'Please answer the user question.'
        }
    ]

options = {
    'guardrails': True,
}

Complete("claude-3-5-sonnet", messages, options = options)

## Put it together

Now we have a single method that can answer questions from tabular data!

In [ ]:
def answer_question_using_analyst(query: str):
    # use cortex analyst to generate sql for the query
    analyst_output = get_sql_from_cortex_analyst(query)
    # execute sql
    sql_output = session.sql(analyst_output['statement'])
    # make the output LLM-readable
    markdown_sql_output = sql_output.to_pandas().to_markdown(index=False)
    # send query and sql results to LLM
    messages = [
        {
            'role': 'system',
            'content': 'You are a helpful assistant that sql output to answer natural language questions.'
        },
        {
            'role': 'user',
            'content': f'The user has posed a question, which is captured in: {user_query}. '
                       f'The question has been translated into a SQL statement, executed, and the results are found in:'
                       f'\n{markdown_sql_output}'
                       f'Please answer the user question.'
        }
    ]
    
    
    options = {
        'guardrails': True,
    }

    response = Complete("claude-3-5-sonnet", messages, options = options)

    return response

In [ ]:
answer_question_using_analyst("What is the highest daily revenue recorded in a single day in each sales region?")